# Technical Factor - Relative Strength Index (RSI)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (m_sum(IF(close > m_lag(close, 1), close, 0), 6) / 6) / ((m_sum(IF(close < m_lag(close, 1), close, 0), 6) / 6) + 0.0001) AS RS,
        100 - (100 / (1 + RS)) AS RSI,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(RSI > 50 AND m_lag(RSI, 1) <= 50, 1, 0) AS TRBY1,
        IF(RSI < 50 AND m_lag(RSI, 1) >= 50, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(RSI > m_lag(RSI, 1) AND m_lag(RSI, 1) >= m_lag(RSI, 2), 1, 0) AS MTBY1,
        IF(RSI < m_lag(RSI, 1) AND m_lag(RSI, 1) <= m_lag(RSI, 2), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(RSI < 30 AND (RSI - m_lag(RSI, 1)) > 0, 1, 0) AS RVBY1,
        IF(RSI > 70 AND (RSI - m_lag(RSI, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(RSI, 1) <= 70 AND RSI > 70, 1, 0) AS BKBY1,
        IF(m_lag(RSI, 1) >= 30 AND RSI < 30, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        RS,
        RSI,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,RS,RSI,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-09,600519.SH,1.484288,59.747020,0,0,0,0,0,0,0,0
1,2025-01-10,600519.SH,0.994415,49.859986,0,1,0,0,0,0,0,0
2,2025-01-13,600519.SH,2.006495,66.738678,1,0,0,0,0,0,0,0
3,2025-01-14,600519.SH,5.043997,83.454657,0,0,1,0,0,0,1,0
4,2025-01-15,600519.SH,1.996024,66.622427,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
233,2025-12-25,600519.SH,0.502159,33.429132,0,0,0,0,0,0,0,0
234,2025-12-26,600519.SH,0.501526,33.401101,0,0,0,0,0,0,0,0
235,2025-12-29,600519.SH,0.201072,16.741049,0,0,0,1,0,0,0,1
236,2025-12-30,600519.SH,0.201604,16.777873,0,0,0,0,1,0,0,0
